In [64]:
import pandas as pd
import numpy as np

train=pd.read_csv('./apa_train.csv')
test=pd.read_csv('./apa_test.csv')
submission=pd.read_csv('./apa_submission.csv')

# , 처리
test.employee1=test.employee1.astype('str').str.replace(",", "").astype('float')
test.employee2=test.employee2.astype('str').str.replace(",", "").astype('float')


# oc 변경
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

train['OC']=le.fit_transform(train['OC'])



In [65]:
# instkind변경
def groupinstkind(x):
    if x=="clinic" or x=="dental_clinic" or x=="traditional_clinic":
        x=1
    elif x=="hospital"or x=="traditional_hospital":
        x=2
    elif x=='general_hospital'or x== 'nursing_hospital':
        x=3
    else:
        None
    return x

train['instkind']=train['instkind'].apply(groupinstkind)
test['instkind']=test['instkind'].apply(groupinstkind)



In [66]:
from sklearn.preprocessing import LabelEncoder

# 시도 군구 변경
encoder=LabelEncoder()
encoder.fit(train['sido'])
train['sido']=encoder.transform(train['sido'])
for label in np.unique(test['sido']):
    if label not in encoder.classes_:
        encoder.classes_=np.append(encoder.classes_,label)
test['sido']=encoder.transform(test['sido'])

encoder2=LabelEncoder()
encoder2.fit(train['sgg'])
train['sgg']=encoder2.transform(train['sgg'])
for label in np.unique(test['sgg']):
    if label not in encoder2.classes_:
        encoder2.classes_=np.append(encoder2.classes_,label)
test['sgg']=encoder2.transform(test['sgg'])


In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    int32  
 2   sido                301 non-null    int32  
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            296 non-null    float64
 6   instkind            300 non-null    float64
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

In [68]:
# bedCount에 결측값이 있는 column 
bednullindex=train.index[train['bedCount'].isnull()]

print(train.groupby('instkind')['bedCount'].median())

for index in bednullindex:
    if train.loc[index,'instkind']==2:
        train.loc[index,'bedCount']=66.0
    elif train.loc[index,'instkind']==1:
        train.loc[index,'bedCount']=0.0
    elif train.loc[index,'instkind']==3:
        train.loc[index,'bedCount']=171.0
    else:
        print("bedCount에 채울 수 있는 결측치 없음")

bednullindex2=test.index[test['bedCount'].isnull()]
for index in bednullindex2:
    if test.loc[index,'instkind']==2:
        test.loc[index,'bedCount']=66.0
    elif test.loc[index,'instkind']==1:
        test.loc[index,'bedCount']=0.0
    elif test.loc[index,'instkind']==3:
        test.loc[index,'bedCount']=171.0
    else:
        print("bedCount에 채울 수 있는 결측치 없음")

instkind
1.0      0.0
2.0     66.0
3.0    171.0
Name: bedCount, dtype: float64
bedCount에 채울 수 있는 결측치 없음
bedCount에 채울 수 있는 결측치 없음


In [69]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    int32  
 2   sido                301 non-null    int32  
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            300 non-null    float64
 6   instkind            300 non-null    float64
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

In [70]:
train.drop(train.index[train['bedCount'].isnull()],axis=0,inplace=True)

In [71]:
em1null=train.index[train['employee1'].isna()]

em2null=train.index[train['employee2'].isnull()]

em12null=[48, 60, 93, 212, 242, 258, 263, 285, 298]

print(em1null,"\n",em2null,"\n",em12null,"\n")

train.loc[em2null]
train.loc[35,'employee2']=51.0
train.loc[62,'employee2']=38.0
train.loc[230,'employee2']=461.0
train.loc[299,'employee2']=100.0
train.loc[em2null]



Int64Index([48, 60, 93, 206, 212, 242, 258, 263, 285, 298], dtype='int64') 
 Int64Index([35, 48, 60, 62, 93, 212, 230, 242, 258, 263, 285, 298, 299], dtype='int64') 
 [48, 60, 93, 212, 242, 258, 263, 285, 298] 



,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
35,58,0,8,145,20130206,132.0,3.0,2.681766e+09,1.802009e+08,2.014580e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,51.0,51.0,change
48,74,1,14,0,19940425,28.0,2.0,8.872427e+09,9.641249e+08,7.638920e+09,...,5.958278e+09,5.956278e+09,4.992758e+09,2.000000e+06,0.000000e+00,3.625952e+09,0.0,NaN,NaN,NaN
60,94,1,5,116,20031201,244.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,96,1,0,58,20170602,128.0,3.0,5.236406e+08,2.306773e+07,8.355775e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,38.0,38.0,NaN
93,140,0,7,105,20070420,203.0,2.0,5.423357e+09,0.000000e+00,5.212041e+09,...,2.733619e+09,1.406195e+08,0.000000e+00,2.590000e+09,2.590000e+09,3.752184e+09,0.0,NaN,NaN,NaN
212,306,0,12,30,19990603,66.0,3.0,2.233031e+10,8.484657e+08,1.849255e+10,...,1.563217e+10,1.288180e+10,0.000000e+00,2.750371e+09,0.000000e+00,1.384958e+10,0.0,NaN,NaN,NaN
230,327,1,12,47,19990309,340.0,3.0,2.115853e+08,0.000000e+00,3.794506e+07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,461.0,461.0,same
242,345,1,0,63,20170420,0.0,1.0,6.845503e+08,0.000000e+00,5.619356e+08,...,6.957960e+07,6.920960e+07,0.000000e+00,3.700000e+05,0.000000e+00,1.900197e+09,0.0,NaN,NaN,NaN
258,363,1,7,88,20141219,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,369,1,5,92,20020123,39.0,2.0,3.007769e+09,1.197756e+09,2.258240e+09,...,1.883430e+09,5.315201e+08,2.000000e+08,1.351910e+09,1.201910e+09,1.017573e+09,226197450.0,NaN,NaN,NaN


In [72]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             300 non-null    int64  
 1   OC                  300 non-null    int32  
 2   sido                300 non-null    int32  
 3   sgg                 300 non-null    int64  
 4   openDate            300 non-null    int64  
 5   bedCount            300 non-null    float64
 6   instkind            300 non-null    float64
 7   revenue1            292 non-null    float64
 8   salescost1          292 non-null    float64
 9   sga1                292 non-null    float64
 10  salary1             292 non-null    float64
 11  noi1                292 non-null    float64
 12  noe1                292 non-null    float64
 13  interest1           292 non-null    float64
 14  ctax1               292 non-null    float64
 15  profit1             292 non-null    float64
 16  liquidAs

In [73]:
print(len(train[train['OC']==1]))
print(len(train[train['OC']==0]))

train1=pd.concat([train[train['OC']==0].copy(),train[train['OC']==0].copy()],ignore_index=True)
train1=pd.concat([train1.copy(),train1.copy()],ignore_index=True)
train1=pd.concat([train1.copy(),train1.copy()],ignore_index=True)
train1=pd.concat([train1.copy(),train1.copy()],ignore_index=True)
train1=pd.concat([train1.copy(),train1.copy()],ignore_index=True)

train=pd.concat([train.copy(),train1.copy()],ignore_index=True)

286
14


In [74]:
test[test['instkind'].isnull()]
test.loc[120,'instkind']=2
test.loc[125,'bedCount']=0
test.loc[125,'instkind']=3

print(test['ownerChange'])



0      same
1      same
2       NaN
3      same
4      same
       ... 
122     NaN
123    same
124     NaN
125    same
126     NaN
Name: ownerChange, Length: 127, dtype: object


In [75]:
traini=[]
for i in train:
    traini.append(i)
print()

features=['inst_id', 'bedCount', 'instkind', 'revenue1', 'salescost1', 'sga1', 'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1', 'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAsset1', 'nonCAsset1', 'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1', 'liquidLiabilities1', 'shortLoan1', 'NCLiabilities1', 'longLoan1', 'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2', 'noi2', 'noe2', 'interest2', 'ctax2', 'profit2', 'liquidAsset2', 'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2', 'tanAsset2', 'OnonCAsset2', 'receivableL2', 'debt2', 'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2', 'netAsset2', 'surplus2']


from sklearn.impute import SimpleImputer
my_imputer=SimpleImputer()
test[features]=my_imputer.fit_transform(test[features])
train[features]=my_imputer.fit_transform(train[features])


X_train, y_train,X_test = train[features],train['OC'],test[features]

X_train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             412 non-null    float64
 1   bedCount            412 non-null    float64
 2   instkind            412 non-null    float64
 3   revenue1            412 non-null    float64
 4   salescost1          412 non-null    float64
 5   sga1                412 non-null    float64
 6   salary1             412 non-null    float64
 7   noi1                412 non-null    float64
 8   noe1                412 non-null    float64
 9   interest1           412 non-null    float64
 10  ctax1               412 non-null    float64
 11  profit1             412 non-null    float64
 12  liquidAsset1        412 non-null    float64
 13  quickAsset1         412 non-null    float64
 14  receivableS1        412 non-null    float64
 15  inventoryAsset1     412 non-null    float64
 16  nonCAss

In [76]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
# RandomForestClassifier 객체 불러오기
rfc=RandomForestClassifier()

rfc_param_grid={
    'n_estimators':[5,7,10,12,15],
    'max_depth':[2,3,4],
    'min_samples_leaf':[2,3,5],
    'min_samples_split':[2,3]
}

grid_rfc=GridSearchCV(rfc,param_grid=rfc_param_grid,n_jobs=-1,verbose=1)
grid_rfc.fit(X_train, y_train)

# 최적의 파라미터와 최고 예측 정확도 계산
print('최적 파라미터      : ', grid_rfc.best_params_)
print('최고 예측 정확도   : {0:.4f}'.format(grid_rfc.best_score_))

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 352 tasks      | elapsed:    3.8s


최적 파라미터      :  {'max_depth': 4, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 7}
최고 예측 정확도   : 0.9369


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:    4.9s finished


In [77]:
# 최적의 파라미터로 학습된 트리 Estimator 가져오기
estimator = grid_rfc.best_estimator_

# 검증 데이터로 결과 예측
pred = estimator.predict(X_test)


In [78]:
sub1=submission.copy()
sub1['OC']=pred
sub1.to_csv('./sub1.csv',index=False)